In [2]:
import sqlite3

conn = sqlite3.connect('rtb_database.db')
cursor = conn.cursor()
create_table_query = """
CREATE TABLE IF NOT EXISTS final_report (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    month TEXT,
    campaign_id INTEGER,
    brand TEXT,
    number_of_clicks INTEGER,
    number_of_transactions INTEGER,
    advertising_campaign_cost REAL,
    value_of_products REAL
);
"""

cursor.execute(create_table_query)
conn.commit()
conn.close()

print("Table 'final_report' with aggregation columns has been created in 'rtb_database.db'")


Table 'final_report' with aggregation columns has been created in 'rtb_database.db'


In [3]:
conn = sqlite3.connect('rtb_database.db')
cursor = conn.cursor()

sql_query = """
WITH MonthlyClicks AS (
    SELECT strftime('%Y-%m', date) AS month, campaign_id, COUNT(*) AS number_of_clicks 
    FROM clicks 
    GROUP BY month, campaign_id
),

MonthlyTransactions AS (
    SELECT strftime('%Y-%m', date) AS month, campaign_id, COUNT(DISTINCT transaction_id) AS number_of_transactions 
    FROM transactions 
    GROUP BY month, campaign_id
),

CampaignCost AS (
    SELECT 
        strftime('%Y-%m', t.date) AS month,
        t.campaign_id,
        SUM(0.01 * c.number_of_clicks + 0.10 * t.transaction_value) AS advertising_campaign_cost
    FROM 
        MonthlyClicks c
    JOIN 
        transactions t ON c.campaign_id = t.campaign_id
    GROUP BY 
        month, t.campaign_id
),

ProductValues AS (
    SELECT 
        strftime('%Y-%m', t.date) AS month,
        p.brand,
        SUM(p.price) AS value_of_products
    FROM 
        transactions t
    JOIN 
        transactions_products tp ON t.transaction_id = tp.transaction_id
    JOIN 
        products p ON tp.product_id = p.id
    GROUP BY 
        month, p.brand
)

INSERT INTO final_report (month, campaign_id, brand, number_of_clicks, number_of_transactions, advertising_campaign_cost, value_of_products)
SELECT 
    mc.month,
    mc.campaign_id,
    pv.brand,
    mc.number_of_clicks,
    mt.number_of_transactions,
    cc.advertising_campaign_cost,
    pv.value_of_products
FROM 
    MonthlyClicks mc
JOIN 
    MonthlyTransactions mt ON mc.campaign_id = mt.campaign_id AND mc.month = mt.month
JOIN 
    CampaignCost cc ON mc.campaign_id = cc.campaign_id AND mc.month = cc.month
JOIN 
    ProductValues pv ON mc.month = pv.month
ORDER BY 
    mc.month, mc.campaign_id, pv.brand;
"""

cursor.execute(sql_query)
conn.commit()
conn.close()

print("Data has been successfully inserted into the 'final_report' table.")


Data has been successfully inserted into the 'final_report' table.


In [ ]:
# Saving the output into CSV

import csv

conn = sqlite3.connect('rtb_database.db')
cursor = conn.cursor()

sql_query = sql_query

results = cursor.execute(sql_query).fetchall()

headers = [i[0] for i in cursor.description]

with open('final_report.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(results)

In [ ]:
df =